In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(style='whitegrid')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("AIML Dataset.csv")

In [ ]:
df.head()

In [ ]:
df.info()       # general info about data types and memory usuage, here we have 7 float, 1 integer and 3 object types

In [ ]:
df.columns

In [ ]:
df['isFraud'].value_counts()

In [ ]:
df['isFlaggedFraud'].value_counts()

In [ ]:
df.isnull().sum().sum()     # isnull() returns cell wise boolean; sum() it returns in column wise

In [ ]:
df.shape  # we have 7...rows,11 columns

In [ ]:
# want to see percentage of fraud to the total data

(df['isFraud'].value_counts()[1]/df.shape[0]) * 100

df['isfraud'] ---> picks the column named isFraud from the dataframe(likely contains 1 for fraud and 0 for not fraud)

value_counts() ---> counts how many 1s and 0s are there in that column

[1] ---> select the counts of rows where isfraud is 1

df.shape[0] ---> gets the total number of rows in the dataframe.

Example:

If your data looks like this:

isFraud
0
1
0
1
0

value_counts() → {0: 3, 1: 2}

So value_counts()[1] → 2 (fraud cases)

df.shape[0] → 5 (total rows)

In [ ]:
round((df['isFraud'].value_counts()[1] / df.shape[0])*100,2)

In [ ]:
# now we'll analyze our data by visualizing imp thing about fraud detection. let's start with visualing transcation types.

df["type"].value_counts().plot(kind='bar', title='Transaction type', color='skyblue')
plt.xlabel('Transactional type')
plt.ylabel('Count')
plt.show()

In [ ]:
# to find fraud rates by type

fraud_by_type = df.groupby('type')['isFraud'].mean().sort_values(ascending=False)
fraud_by_type.plot(kind='bar', title='Fraud rate by type', color='salmon')
plt.ylabel('Fraud rate')
plt.show()

In [ ]:
fraud_by_type